In [ ]:
import ct.mlexp2 as ml
from importlib import reload
from datetime import date
from cdcqr.common.utils import load_df, save, resample_pv
from cdcqr.data.glassnode.utils import get_glassnode_data, quantile_info, url_parser
import time
from IPython.display import Audio
from cdcqr.common.config import GLASSNODE_API_KEY
sound_file = '/tmp/ding.wav'
import sys
sys.path.append('/github/cryptoderiv-quant/')
from ct.utils import glassnode


### backtest on Transactions data

In [ ]:
df_dict = {}
data_sym = 'BTCUSD_PERP@binance'
df_dict['BTC'] = load_df(data_sym).resample_pv('10Min')

data_sym = 'ETHUSD_PERP@binance'
df_dict['ETH'] = load_df(data_sym).resample_pv('10Min')

In [ ]:
category = 'Transactions'
transactions_features_exchange_activity = ['TransfersVolumeToExchangesSum',
                                             'TransfersVolumeFromExchangesSum',
                                             'TransfersVolumeExchangesNet',
                                             'TransfersToExchangesCount',
                                             'TransfersFromExchangesCount',
                                             'TransfersVolumeToExchangesMean',
                                             'TransfersVolumeFromExchangesMean',
                                             'TransfersVolumeWithinExchangesSum',
                                             'TransfersBetweenExchangesCount',
                                             'TransfersVolumeBetweenExchangesSum']
transactions_features_exchange_activity1 = ['TransfersVolumeToExchangesSum',                       
                                             'TransfersVolumeExchangesNet',
                                             'TransfersToExchangesCount',
                                             'TransfersVolumeToExchangesMean',
                                             'TransfersVolumeWithinExchangesSum',
                                             'TransfersBetweenExchangesCount',
                                             'TransfersVolumeBetweenExchangesSum']

for feature_name in transactions_features_exchange_activity1:
    print('-----------------------{}---------------------------------------------------------------'.format(feature_name))
    for underly in ['BTC', 'ETH']:
        print('------------------------{}----------------------------------------------------------------'.format(underly))
        url = 'a={}&category={}&m=transactions.{}'.format(underly, category, feature_name)
        att_dict = url_parser(url)
        category = att_dict['category']
        feature = att_dict['feature']
        underly = att_dict['a']

        freq='10m'
        df_i = df_dict[underly]
        start_time = df_i.index[0].isoformat()
        url1 = 'metrics~{}~{}'.format(category, feature)
        print(url1)
        dfgsi = glassnode(url1,a=underly,i=freq,s=start_time,price=False)
        df_i = df_i[df_i.index<='20211031']
        dff = dfgsi.reindex(df_i.index)
        #dff['v'] = dff[feature]
        t0 = time.time()
        display(dff.head())
        res = ml.mlexp2(dff, df_i, sym=sym,
              qsuffix='USDT@binancealt',
              ytrain_thresh=6,
              ytrain_periods=100,
              ytest_thresh=3,
              ytest_periods=100,
              vdelay=2,
              ytypes=["ys"],
              xtypes=["v"],
              qlags=1000,
              request=['vbt','vbtplot'],
              #models=["oneiflarger","dt"],
              models=["oneiflarger", 'dt'],
              randcv_n_iter=30,
              sl=2,
              tp=6,
              ts=None,
              vbtfreq="10Min") 
        t1 = time.time()
        print(t1-t0)
        Audio(sound_file, autoplay=True)

In [ ]:
category = 'OTC'

transactions_features_otc_desks = ['TransfersVolumeToOtcDesksSum', 
                                        'TransfersVolumeFromOtcDesksSum', 
                                        'TransfersToOtcDesksCount', 
                                        'TransfersFromOtcDesksCount', 
                                        ]

for feature_name in transactions_features_otc_desks:
    print('-----------------------{}---------------------------------------------------------------'.format(feature_name))
    for underly in ['BTC']:
        print('------------------------{}----------------------------------------------------------------'.format(underly))
        url = 'a={}&category={}&m=transactions.{}'.format(underly, category, feature_name)
        att_dict = url_parser(url)
        category = att_dict['category']
        feature = att_dict['feature']
        underly = att_dict['a']

        freq='10m'
        df_i = df_dict[underly]
        start_time = df_i.index[0].isoformat()
        url1 = 'metrics~{}~{}'.format(category, feature)
        print(url1)
        dfgsi = glassnode(url1,a=underly,i=freq,s=start_time,price=False)
        df_i = df_i[df_i.index<='20211031']
        dff = dfgsi.reindex(df_i.index)
        #dff['v'] = dff[feature]
        t0 = time.time()
        display(dff.head())
        res = ml.mlexp2(dff, df_i, sym=sym,
              qsuffix='USDT@binancealt',
              ytrain_thresh=6,
              ytrain_periods=100,
              ytest_thresh=3,
              ytest_periods=100,
              vdelay=2,
              ytypes=["ys"],
              xtypes=["v"],
              qlags=1000,
              request=['vbt','vbtplot'],
              #models=["oneiflarger","dt"],
              models=["oneiflarger", 'dt'],
              randcv_n_iter=30,
              sl=2,
              tp=6,
              ts=None,
              vbtfreq="10Min") 
        t1 = time.time()
        print(t1-t0)
        Audio(sound_file, autoplay=True)